In [30]:
import numpy as np
import seaborn as sb

In [50]:
class Paciente():
    def __init__(self,prefijo,tipo):
        self.prefijo = prefijo
        self.tiempoLlegada = round(np.random.poisson(220)) #llegada de pacienes con media=220 y poisson
        self.tipo = tipo
        if tipo == "internacion":
            self.duracionInternacion = round(np.random.uniform(1,3)) 
        else:
            self.duracionInternacion = round(np.random.uniform(2,5)) #2 y 5 días para pacientes operados

In [99]:
class Cama():
    def __init__(self, id):
        self.id = id
        self.ocupada = False
    
    def marcarOcupada():
        self.ocupada = True
    
    def marcarDesocupada():
        self.ocupada = False

In [100]:
class Quirofano():
    def __init__(self,id):
        self.id = id
        self.ocupado = False
        self.abierto = True
    
    def marcarOcupado():
        self.ocupado = True
    
    def marcarDesocupado():
        self.ocupado = False

In [101]:
class Evento():
    def __init__(self, _id, nombre, objeto, arribo, duracion):
        self.id = _id
        self.nombre = nombre
        self.arribo = arribo
        self.duracion = duracion
        self.tiempo_fin = arribo + duracion
        self.objeto = objeto
        
    def __repr__(self):
        return f"{self.id} - {self.nombre} - arribo: {self.arribo} - duracion: {self.duracion} - termina: {self.tiempo_fin}\n"
    
    
class EventoFinAtencion(Evento):
    def __init__(self, evento_inicio, **kwargs):
        super(EventoFinAtencion, self).__init__(**kwargs)
        self.evento_inicio = evento_inicio
    
    def __repr__(self):
        return f"{self.id} - {self.nombre} a {self.evento_inicio.id} - inicia: {self.arribo} - duracion: {self.duracion} - termina: {self.tiempo_fin}\n"   

In [102]:
def hayCamaLibre(lista_camas):
    """
    Consulta si hay una cama libre
    """
    return any(not cama.ocupada for cama in lista_camas)

In [103]:
def hayQuirofanoLibre(lista_quirofanos):
    """
    Consulta si hay un quirofano libre
    """
    return any(not quirofano.ocupado for quirofano in lista_quirofanos)

In [104]:
def ordenarEventos(fel):
    """
    Ordena la lista de eventos por tiempod de arribo
    """
    fel.sort(key=lambda evento: evento.arribo)
    return fel

In [105]:
def removerEvento(fel, evento):
    """
    remueve el siguiente evento de la FEL
    """
    if evento in fel:
        fel.remove(evento)
    return ordenarEventos(fel)

In [106]:
def crearFEL(paciente, cantidad_pacientes):
    """
    Crea la FEL con eventos de arribo paciente segun el tipo
    de paciente y la cantidad de pacientes que arriban
    """
    lista_eventos = []
    for i in range(cantidad_pacientes):
        lista_eventos.append(Evento(paciente.prefijo, "ARRIBO_PACIENTE", paciente, paciente.tiempoLlegada, 0))
    return ordenarEventos(lista_eventos)

In [107]:
def inicializarModelo(prefijo, cantidad_corridas, cantidad_pacientes, cantidad_quirofanos, cantidad_camas):
    """
    En este caso corrida va a ir a la par con el reloj
    """
    fel = {}
    lista_camas = []
    lista_quirofanos = []
    
    #Inicializamos la fel
    for corrida in range(cantidad_corridas):
        for id_paciente in range(cantidad_pacientes): 
            tipoPaciente = None
            indicador = round(np.random.uniform(0,100))
            if (indicador <= 20): #el 20% de los pacientes vienen a operarse
                tipoPaciente = "operacion"
            else: #el restante 80% solo viene a internarse
                tipoPaciente= "internacion"
            paciente = Paciente(f"{prefijo}-C{id_paciente}",tipoPaciente)
            fel[corrida] = crearFEL(paciente, cantidad_pacientes)

    #Inicializamos los quirofanos
    for i in range(cantidad_quirofanos):
        lista_quirofanos.append(Quirofano(i))
        
    #Inicializamos las camas
    for i in range(cantidad_camas):
        lista_camas.append(Cama(i))
        
    return fel, lista_quirofanos, lista_camas

In [108]:
def asignarCama(eventos):
    pass

In [109]:
def asignarQuirofano(eventos):
    pass

In [ ]:
def getEventosPorCondicion(lista_eventos):
    """
    Esta funcion sirve para determinar cuales eventos de una lista de eventos 
    en un instante de reloj son para ser realizados en las actividades de 
    quirofano libre o cama libre.
    """
    eventos_internarse=[]
    eventos_operarse = []
    eventos_fin_internacion = []
    eventos_fin_operacion = []

    for evento in lista_eventos:
        if (evento.objeto.tipo == 'internacion'):
            eventos_cama_libre.append(evento)
        elif:
            pass #TODO: seguir implementando
    
    return eventos_internarse, eventos_operarse, eventos_fin_internacion, eventos_fin_operacion


In [128]:
def simulacion(cant_experimentos, cant_corridas, cant_pacientes, cant_quirofanos, cant_camas):
    # Esqueleto para simulación con escaneo de actividades
    delta = 1 # Incremento para el valor del reloj
    tope_reloj = 24*60 # Definimos un punto para cortar la ejecución, en este caso 24 hs (en minutos)
    
    
    
    for i in range(cant_experimentos):
        for j in range(cant_corridas):
            prefijo = f"{i}{j}" # un prefijo para identificar los eventos de cada experimento/corrida
            fel, lista_quirofanos, lista_camas = inicializarModelo(prefijo, cant_corridas, cant_pacientes, cant_quirofanos, cant_camas)
            reloj = 0 # Iniciamos/reiniciamos el reloj
            while reloj <= tope_reloj or len(fel) != 0:
                # TODO: checkear el horario de apertura del hospital

                # Obtenemos todos los eventos diferenciados desde la lista de eventos
                eventos_internarse,eventos_operarse, eventos_fin_internacion, eventos_fin_operacion = getEventosPorCondicion(fel)
                
                # Si hay eventos de finalizacion los hago primero para poder seguir procesando nuevas llegadas
                if (eventos_fin_internacion or eventos_fin_operacion):
                    for evento in eventos_fin_internacion:
                        cama = evento.objeto
                        cama.marcarDesocupada() #si evento es fin de internacion el objeto es una cama

                    for evento in eventos_fin_operacion:
                        quirofano = evento.objeto
                        quirofano.marcarDesocupado() #si evento es fin de operacion el objetoes un quirofano
                    
                if hayCamaLibre(lista_camas):
                    # Procesamos eventos que ocurren en este momento de reloj
                    # y que puedan procesarse bajo esta condicion
                    print(".............")
                    print(fel[reloj])
                    print(".............")
                    eventos = fel[reloj]
                    
                    # Ejecutamos actividad con todos los eventos
                    for e in eventos:
                        asignarCama(eventos)
                        # Calculamos estadisticas si es necesario

                    # Retiramos eventos procesados de la fel para el instante correspondiente
                    fel.pop(reloj, None)
                    # Posiblemente generemos nuevos eventos para la fel
                    
                if hayQuirofanoLibre(lista_quirofanos):
                    
                    # ...
                    eventos = fel[reloj]
                    # Ejecutamos actividad con todos los eventoss
                    for e in eventos:
                        asignarQuirofano(eventos)
                        # Calculamos estadisticas si es necesario

                    # Retiramos eventos procesados de la fel para el instante correspondiente
                    fel.pop(reloj, None)
                    # Posiblemente generemos nuevos eventos para la fel
                #...
                
                

                reloj += delta # Avanzamos el reloj al siguiente incremento.

            # Finalizó el procesamiento para esta corrida, hacemos la estadistica necesaria por corrida aqui

        # Finalizó el procesamiento para este experimento, hacemos la estadistica necesaria por experimento aqui

    # Finalizaron los experimentos, hacemos las estadísticas finales, gráficos y reportes aquí

IndentationError: expected an indented block (<ipython-input-128-38b9e6d29f6f>, line 21)

In [127]:
simulacion(cant_experimentos=1, cant_corridas=30, cant_pacientes=20, cant_quirofanos=4, cant_camas=10)

.............
[00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C19 - ARRIBO_PACIENTE - arribo: 204 - duracion: 0 - termina: 204
, 00-C1

KeyError: 0